In [17]:
# pip install playwright

In [18]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

In [28]:
async def scrape_jobstreet():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://ph.jobstreet.com/Data-Science-jobs?jobId=85753657&type=standard")

        print("🛑 Manually solve CAPTCHA in browser window if needed...")
        await asyncio.sleep(30)

        jobs = await page.query_selector_all('article[data-testid="job-card"]')
        print(f"🔍 Found {len(jobs)} job listings")

        data = []

        for job in jobs:
            try:
                # Get title from article's aria-label
                title = await job.get_attribute("aria-label")
                title = title.strip() if title else "N/A"

                # Job URL
                anchor = await job.query_selector("a[data-automation='job-list-item-link-overlay']")
                url_suffix = await anchor.get_attribute("href") if anchor else "N/A"
                job_url = "https://ph.jobstreet.com" + url_suffix if url_suffix and url_suffix.startswith("/job") else url_suffix

                # Get all span text for company/location/salary/posted
                all_spans = await job.query_selector_all("span")
                text_items = []
                for span in all_spans:
                    try:
                        txt = await span.inner_text()
                        if txt.strip():
                            text_items.append(txt.strip())
                    except:
                        continue

                # Use heuristics to extract fields
                company = text_items[0] if len(text_items) > 0 else "N/A"
                location = next((t for t in text_items if "Metro Manila" in t or "City" in t), "N/A")
                salary = next((t for t in text_items if "₱" in t or "PHP" in t), "N/A")
                posted = next((t for t in text_items if "ago" in t or "day" in t or "hour" in t), "N/A")

                data.append({
                    "Title": title,
                    "Company": company,
                    "Location": location,
                    "Salary": salary,
                    "Posted": posted,
                    "URL": job_url
                })

            except Exception as e:
                print("⚠️ Error parsing a job:", e)

        await browser.close()
        return pd.DataFrame(data)

In [29]:
# Run the async function inside Jupyter
df = await scrape_jobstreet()
df.head()

🛑 Manually solve CAPTCHA in browser window if needed...
🔍 Found 32 job listings


,Title,Company,Location,Salary,Posted,URL
0,Data Analyst,at\nNational Data and Surveying Services Inc,"Pasig City, Metro Manila","₱25,000 – ₱35,000 per month",1d ago,https://ph.jobstreet.com/job/85753657?type=sta...
1,Analytics Engineer - US WFH,"at\nHelloConnect, Inc.","Quezon City, Metro Manila(Remote)","₱75,000 – ₱100,000 per month",11h ago,https://ph.jobstreet.com/job/85795350?type=sta...
2,Data Analyst,"at\nRiteMed Phils., Inc.","Mandaluyong City, Metro Manila(Hybrid)",N/A,1d ago,https://ph.jobstreet.com/job/85756366?type=sta...
3,Data Scientist (Dayshift - WFH),at\nMicroSourcing,Metro Manila(Remote),"₱120,000 per month",2d ago,https://ph.jobstreet.com/job/85735664?type=sta...
4,Data Analyst,at\nTHOMSON REUTERS CORP PTE LTD - PHILIPPINE ...,"Taguig City, Metro Manila",N/A,8d ago,https://ph.jobstreet.com/job/85550130?type=sta...


In [30]:
display(df)

,Title,Company,Location,Salary,Posted,URL
0,Data Analyst,at\nNational Data and Surveying Services Inc,"Pasig City, Metro Manila","₱25,000 – ₱35,000 per month",1d ago,https://ph.jobstreet.com/job/85753657?type=sta...
1,Analytics Engineer - US WFH,"at\nHelloConnect, Inc.","Quezon City, Metro Manila(Remote)","₱75,000 – ₱100,000 per month",11h ago,https://ph.jobstreet.com/job/85795350?type=sta...
2,Data Analyst,"at\nRiteMed Phils., Inc.","Mandaluyong City, Metro Manila(Hybrid)",N/A,1d ago,https://ph.jobstreet.com/job/85756366?type=sta...
3,Data Scientist (Dayshift - WFH),at\nMicroSourcing,Metro Manila(Remote),"₱120,000 per month",2d ago,https://ph.jobstreet.com/job/85735664?type=sta...
4,Data Analyst,at\nTHOMSON REUTERS CORP PTE LTD - PHILIPPINE ...,"Taguig City, Metro Manila",N/A,8d ago,https://ph.jobstreet.com/job/85550130?type=sta...
5,Data Scientist (Fraud Analytics),at\nTransUnion Information Solutions Inc.,"Makati City, Metro Manila(Hybrid)",N/A,7d ago,https://ph.jobstreet.com/job/85603671?type=sta...
6,Data Analyst (Virtual Assistant),"at\nCover Desk Philippines, Inc.","Quezon City, Metro Manila","₱30,000 – ₱35,000 per month",7d ago,https://ph.jobstreet.com/job/85593540?type=sta...
7,URGENT | Senior Data Scientist | Alabang,at\nGenpact Services LLC,"Muntinlupa City, Metro Manila",N/A,1d ago,https://ph.jobstreet.com/job/85782442?type=sta...
8,Senior Data Scientist,at\nEY GLOBAL DELIVERY SERVICES (GDS) PHILIPPINES,Metro Manila(Hybrid),N/A,3d ago,https://ph.jobstreet.com/job/85700899?type=sta...
9,Analytics Officer,at\nSecurity Bank Corporation,"Makati City, Metro Manila(Hybrid)",N/A,HMO coverage from day one,https://ph.jobstreet.com/job/85795523?type=sta...
